# Compartimental Model Simulator

**Created by:** Samuel Ropert  
**Creation date:** 04/08/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how to build a single SEIR simulation object with different quarantine scenarios. In this example we show 4 quarantine scenarios, 2 with total quarantines, and 2 with dynamic quarantines.


## Import Libraries

In [47]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, 'src/SEIR/')
from SEIRmodel import SEIRmodel
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
from joblib import Parallel, delayed
import multiprocessing

## Geografical Parameters
Elegir comuna o region según su código CUT

In [48]:
tstate = '13' # tstate = ['13','14']
initdate = datetime(2020,5,15)

## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

In [79]:
beta = 0.12 # Contagion rate
mu = 0.5 # E0/I0 initial rate

k=40 # Kinetic Saturation: 0 for mass action mixing

ScaleFactor = 1 # Scale Factor: Number of real infected over reported 
SeroPrevFactor = 1 # Sero Prevalence Factor: Adjust the proportion of the population that enters the virus dynamics
expinfection = 1 # Exposed contagion rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [80]:
# Simulation time
tsim = 1000
# Population in case we don't pick a specific place
if not tstate:
    population = 1000000
# Initial Active Infected 
I_act0 = 100


### Quarantines

Quarantine Vector:  
 [Tsim, max_mob,rem_mob,quarantine period, quarantine initial time, quarantine final time, quarantine type]

In [81]:
max_mob = 0.8 # Maximum mobility
# Total quarantine
s1 = [tsim,max_mob,0.65,0,0,tsim,0]
s2 = [tsim,max_mob,0.5,0,0,tsim,0]
# Dynamic quarantine
s3 = [tsim,max_mob,0.3,14,0,tsim,1]
s4 = [tsim,max_mob,0.5,14,0,tsim,1]


In [82]:
# Define one quarantine array for each different quarantine remanent mobility
quarantines = [s1,s3]

## Create simulation Object

In [83]:
simulation = SEIRmodel(beta = beta,mu = mu,ScaleFactor=ScaleFactor,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,tsim = tsim,tstate=tstate, k = k,initdate=initdate)
simulation.inputarray = np.array(quarantines)
simulation.addquarantine()
if not tstate:
    simulation.initialvalues(I_act0,population,R=0)

Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done


# Paralelización
1. Desacoplar objeto de datos externos, si no hay que importar los datos cada vez
2. Implementar dinámicas de paralelización para iterar con distintos parámetros (beta, k,mu, cuarentenas,seroprev, etc)
3. Agregar infectados acumulados al valor inicial
4.

In [88]:
sims = []
k = [0,5,10,15,20,30,40]
for i in k:
    simulation = SEIRmodel(beta = beta,mu = mu,ScaleFactor=ScaleFactor,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,tsim = tsim,tstate=tstate, k = i,initdate=initdate)
    simulation.inputarray = np.array(quarantines)
    simulation.addquarantine()
    if not tstate:
        simulation.initialvalues(I_act0,population,R=0)
    sims.append(simulation)

Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done
Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done
Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done
Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done
Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochimi
Import Population
Infectados Activos
Infectados diarios
Infectados Acumulados
Done
Importando Datos
Fallecidos Acumulados
Fallecidos Excesivos
Infectados Activos Minciencia
Sochi

## Simulate
The different scenarios are simulated in parallel threads 

In [84]:
simulation.simulate()

SEIR Model
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done   2 out of   2 | elapsed:   30.4s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   2 out of   2 | elapsed:   30.4s finished


../SEIR/SEIR_vars.py:143: RuntimeWarning: invalid value encountered in double_scalars
  self.SHFR = [self.totD[i]/(self.I_se_ac[i][-1]+self.I_cr_ac[i][-1]) for i in range(self.numescenarios)]
../SEIR/SEIR_vars.py:144: RuntimeWarning: invalid value encountered in true_divide
  self.SHFR_d = [self.B[i]/(self.I_se_ac[i][-1]+self.I_cr_ac[i][-1]) for i in range(self.numescenarios)]


# Simulation Analysis

## Plots
This libraries have predefined plot functions which plot the main epidemiological variables.
Each function has the following optional arguments:
* days [int] Number of days to display
* showparams [bool] Display simulation parameters 
* ylim [int] Limit the vertical axis
* norm [int/float] Normalize the results


In [85]:
enddate =  datetime(2020,8,10)

In [86]:
# Accumulated infected
simulation.plotAccumulatedInfected(reales=True,days=0,enddate=enddate)

In [55]:
# Epidemiological curves
simulation.plotseir(days = 500)

In [19]:
# Active Infected curves
simulation.plotActiveInfected(showparams=True,days = 0, reales=True)

In [20]:
# Daily infected
simulation.plotDailyInfected(days=100,reales=True)

In [21]:
# Exposed
simulation.plotExposed()

In [22]:
# Exposed
simulation.plotQuarantines(days=100)

In [35]:
simulation.Ir_dates

[datetime.datetime(2020, 5, 15, 0, 0),
 datetime.datetime(2020, 5, 18, 0, 0),
 datetime.datetime(2020, 5, 22, 0, 0),
 datetime.datetime(2020, 5, 25, 0, 0),
 datetime.datetime(2020, 5, 29, 0, 0),
 datetime.datetime(2020, 6, 1, 0, 0),
 datetime.datetime(2020, 6, 5, 0, 0),
 datetime.datetime(2020, 6, 8, 0, 0),
 datetime.datetime(2020, 6, 12, 0, 0),
 datetime.datetime(2020, 6, 15, 0, 0),
 datetime.datetime(2020, 6, 19, 0, 0),
 datetime.datetime(2020, 6, 23, 0, 0),
 datetime.datetime(2020, 6, 28, 0, 0),
 datetime.datetime(2020, 7, 1, 0, 0),
 datetime.datetime(2020, 7, 5, 0, 0),
 datetime.datetime(2020, 7, 10, 0, 0),
 datetime.datetime(2020, 7, 13, 0, 0),
 datetime.datetime(2020, 7, 17, 0, 0),
 datetime.datetime(2020, 7, 20, 0, 0),
 datetime.datetime(2020, 7, 24, 0, 0),
 datetime.datetime(2020, 7, 27, 0, 0),
 datetime.datetime(2020, 7, 31, 0, 0),
 datetime.datetime(2020, 8, 3, 0, 0)]

# Saturation Kinetics Dynamics:
The simulation object contains several methods and variables with its results

In [19]:
# Parallel Simulation function definition
num_cores = multiprocessing.cpu_count() 
def ParallelSimulation(quarantines,k=0):     
    #quarantines = [[tsim, 0.85, alpha, qp, iqt, fqt, qt]] 
    simulation = SEIRmodel(beta = beta,mu = mu,ScaleFactor=ScaleFactor,SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,tsim = tsim,tstate=tstate, k = k,initdate=initdate)
    simulation.inputarray = np.array(quarantines)
    simulation.addquarantine()
    if not tstate:
        simulation.initialvalues(I_act0,population,R=0)
    simulation.simulate()  
    return simulation

In [20]:
k = [0,5,10,15,20,25,30,35,40]#list(np.arange(0,kmax+step,step))


In [24]:
# Run  Parallel Simulation s
sims = [] 
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(ParallelSimulation)(quarantines=s1,k=i) for i in k)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done   2 out of   9 | elapsed:    2.4s remaining:    8.3s
[Parallel(n_jobs=8)]: Done   3 out of   9 | elapsed:    2.4s remaining:    4.7s
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:    2.4s remaining:    3.0s
[Parallel(n_jobs=8)]: Done   5 out of   9 | elapsed:    2.4s remaining:    1.9s
[Parallel(n_jobs=8)]: Done   6 out of   9 | elapsed:    2.4s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   7 out of   9 | elapsed:    2.4s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:    2.4s remaining:    0.0s


IndexError: too many indices for array

In [ ]:
# Peak Values per each scenario
simulation.peak

In [ ]:
# Peak time
simulation.peak_t

In [ ]:
simulation.showscenarios()

In [12]:
tstate

'13101'